In [1]:
#def init():
    #CHANGE THIS TO YOUR TRAIN FOLDER
image_folder = 'C:/Study/Semester2/Machine Learning/ML_images/Training' 
#image_folder = 'C:/Users/danie/Desktop/Training_ML'

TOO_SMALL = 200000
SMALL = 500000
MEDIUM = 1200000
LARGE = 2100000
    
X_train = []
Y_train = []
counter = 1
    
img_height = 1888 #Usually 128. this might change depending on Annaya & Danielle's input 
img_width = 1888 #Usually 128. this might change depending on Annaya & Danielle's input 
epochs = 10 #Start with 1, and increase to 10, 100, 500, 1000 and 3000. 11 is the recommended number of runs through the training dataset. We will probably have to tune this. 
#total_training = total number of training data len()
batch_size = 1 #test different ones from 1 to 2, 5, 10, etc
total_images = 1

In [2]:
#pip install opencv-python

In [3]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import skimage
import cv2
import glob
import pandas as pd
import joblib

from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, UpSampling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage import data, io,color
from skimage.io import imread_collection #loads a collection of images
from skimage.io import imread, imshow
from pathlib import Path

from sklearn.decomposition import PCA

import matplotlib as mpl
import matplotlib.pyplot as plt

In [4]:
#Define sizes of images based on their resolution
#Image shape and resolution

#TODO make the IMG sizes make sense


def shape(img):
    size = 0
    resolution = img.shape[0] * img.shape[1]
    if resolution <= TOO_SMALL:
        size = 0
    elif resolution <= SMALL:
        size = 1
    elif resolution <= MEDIUM:
        size = 2
    elif resolution <= LARGE:
        size = 3
    else:
        size = 6
    return size

In [5]:
def create_dataframe(image_folder):
    images = Path(image_folder).glob('*.jpg')

    list_of_files = []
    for image in images:
        list_of_files.append(str(image))

    df = pd. DataFrame(list_of_files, columns = ["image_path"])
    df_length = len(df.index)
    print("processing", df_length, "photos for training")
    
    height= []
    width = []
    resolution = []
    
    for image_path in df['image_path']:
        img = cv2.imread(str(image_path))
        height.append(img.shape[0])
        width.append(img.shape[1])
        
    df["height"] = height
    df["width"] = width
    df["resolution"] = df["height"] * df["width"]
    
    df = df[df.resolution < LARGE]
    
    ll = df_length - len(df.index)
    print("dropped ", ll, "photos because they were too large to process")
    
    df = df[df.resolution > TOO_SMALL]
    
    ss = df_length - len(df.index) - ll
    print("dropped ", ss, "photos because they were too small to process")
    df.head()
    
    return df

In [6]:
#PCA rgb then pass to prep_img as an rgb image

def pca_rgb(imgBGR, size): #imgRGB, size
    b,g,r = cv2.split(imgBGR)
    list_channels = [b,g,r]
    
    #check size first
    if size ==1:
        inverted_img = []
        for c in list_channels:            
            pca = PCA(n_components = 100)
            transformed = pca.fit_transform(c/255) #r_scaled
            inverted = pca.inverse_transform(transformed)
            inverted_img.append(inverted)
        
        tuple(inverted_img)
        RGB_compressed = cv2.merge(inverted_img)    

    elif size == 2:
        inverted_img = []
        for c in list_channels:            
            pca = PCA(n_components = 300)
            transformed = pca.fit_transform(c/255) #r_scaled
            inverted = pca.inverse_transform(transformed)
            inverted_img.append(inverted)
        
        tuple(inverted_img)
        RGB_compressed = cv2.merge(inverted_img)
    else:
        inverted_img = []
        for c in list_channels:            
            pca = PCA(n_components = 500)
            transformed = pca.fit_transform(c/255) #r_scaled
            inverted = pca.inverse_transform(transformed)
            inverted_img.append(inverted)
        
        tuple(inverted_img)
        RGB_compressed = cv2.merge(inverted_img)
        
    RGB_compressed = np.float32(RGB_compressed)
        
    return RGB_compressed

In [7]:
# image padding

def rgb_pad(image):
    
    height = image.shape[0]
    width = image.shape[1]

    WHITE = [255,255,255] #[255,255,255]  1, 1, 1
    MAX = 1888
    
    if height % 2 == 0:
        top = int((MAX - height)/ 2)
        bottom = int((MAX - height)/ 2)
    else: 
        top = int((MAX - (height-1))/ 2)
        bottom = int(((MAX - (height-1))/ 2)-1)

    if width % 2 == 0:
        left = int((MAX - width)/ 2)
        right = int((MAX - width)/ 2)
    else: 
        left = int((MAX - (width-1))/ 2)
        right = int(((MAX - (width-1))/2)-1)
    
    image = cv2.copyMakeBorder(image, top, bottom, left, right,
    cv2.BORDER_CONSTANT,value=WHITE)

    return image

In [8]:
#Debugged function
def prep_img(img_path):
    imgRGB = cv2.imread(img_path)
    imgBGR = np.flip(imgRGB, -1)
    
    #check the image category by 'shape' function
    image_size_category = shape(imgBGR) #imgRGB
    #print("image size determined!")

    if image_size_category <1:
        raise Exception("Your image is too small to be used")
    elif image_size_category >4:
        raise Exception("Your image is too big to be used")

    #print("image PCA started!")
    #image_pca = pca_rgb(imgRGB, image_size_category) 
    RGB_compressed = pca_rgb(imgBGR, image_size_category)
    RGB_compressed = np.clip(RGB_compressed, 0, 1)
    print('RGB compressed :',np.min(RGB_compressed),np.max(RGB_compressed))
    #plt.figure(figsize=(10, 10))
    #plt.imshow(RGB_compressed)
    
    print("image is padded")
    padded_RGB = rgb_pad(RGB_compressed)
    print(padded_RGB.shape)
    padded_RGB = np.float32(padded_RGB*(1/255))
    print('padded_RGB:', np.min(padded_RGB),np.max(padded_RGB))
   
    #plt.figure(figsize=(10, 10))
    #plt.imshow(padded_RGB*255)
    
    print("image converted to LAB")
    #padded_LAB = cv2.cvtColor(padded_RGB, cv2.COLOR_RGB2Lab) #convert padded RGB compressed to L
    
    padded_LAB = color.rgb2lab(padded_RGB)
    
    print('padded_LAB:', np.min(padded_LAB),np.max(padded_LAB))
    print(padded_LAB.shape)
    #padded_LAB = padded_LAB/np.amax(padded_LAB) #scaled to -1/1
   

    
    #separate L and AB color channels
    Y = padded_LAB[:, :,1:]
    Y = Y *128
    print('A :',np.min(Y[:,:,0]), np.max(Y[:,:,0]))
    print('B :',np.min(Y[:,:,1]), np.max(Y[:,:,1]))
    
    X = padded_LAB[:, :, 0] #X is in a color scale 0 to 100
    print('X :',np.min(X), np.max(X))
    
    #this is in sample code and idk why
    
    X = X.reshape(1, 1888, 1888, 1) #comment out when show predicted
    Y = Y.reshape(1, 1888, 1888, 2) #comment out when show predicted
    
    print("image processed")

    return Y, X

In [9]:
def prep_dataframe(df):
    for index, row in df.iterrows():
        Y, X = prep_img(row["image_path"])
        Y_train.append(Y)
        X_train.append(X)

# Use the code below to process ALL photos and feed them to the model!!!

In [10]:
df = create_dataframe(image_folder)
prep_dataframe(df)

#add padd(image).tolist()
#caused TypeError: unsupported operand type(s) for /: 'list' and 'int'


processing 151 photos for training
dropped  9 photos because they were too large to process
dropped  5 photos because they were too small to process
RGB compressed : 0.0071007092 0.9866514
image is padded
(1888, 1888, 3)
padded_RGB: 2.7845921e-05 1.0
image converted to LAB
padded_LAB: -0.12182295 100.0
(1888, 1888, 3)
A : -13.72242 14.837265
B : -15.593338 14.191818
X : 0.0036582947 100.0
image processed
RGB compressed : 0.0 1.0
image is padded
(1888, 1888, 3)
padded_RGB: 0.0 1.0
image converted to LAB
padded_LAB: -0.16153604 100.0
(1888, 1888, 3)
A : -20.676613 29.351234
B : -17.64412 35.089874
X : 0.0 100.0
image processed
RGB compressed : 0.0 1.0
image is padded
(1888, 1888, 3)
padded_RGB: 0.0 1.0
image converted to LAB
padded_LAB: -0.16416907 100.0
(1888, 1888, 3)
A : -11.717796 11.980057
B : -21.013641 14.585495
X : 0.0 100.0
image processed
RGB compressed : 0.0 1.0
image is padded
(1888, 1888, 3)
padded_RGB: 0.0 1.0
image converted to LAB
padded_LAB: -0.09304285 100.0
(1888, 1888

padded_LAB: -0.1540631 100.0
(1888, 1888, 3)
A : -19.720078 23.676872
B : -15.690231 11.978912
X : 0.0 100.0
image processed
RGB compressed : 0.0 1.0
image is padded
(1888, 1888, 3)
padded_RGB: 0.0 1.0
image converted to LAB
padded_LAB: -0.12775064 100.0
(1888, 1888, 3)
A : -11.618614 9.71508
B : -16.352081 17.661285
X : 0.0 100.0
image processed
RGB compressed : 0.0 1.0
image is padded
(1888, 1888, 3)
padded_RGB: 0.0 1.0
image converted to LAB
padded_LAB: -0.033065677 100.0
(1888, 1888, 3)
A : -3.7031174 11.118889
B : -4.2324066 9.754944
X : 4.5776367e-05 100.0
image processed
RGB compressed : 0.0 1.0
image is padded
(1888, 1888, 3)
padded_RGB: 0.0 1.0
image converted to LAB
padded_LAB: -0.17369539 100.0
(1888, 1888, 3)
A : -22.23301 23.118973
B : -11.355209 21.115494
X : 0.0 100.0
image processed
RGB compressed : 0.0 1.0
image is padded
(1888, 1888, 3)
padded_RGB: 0.0 1.0
image converted to LAB
padded_LAB: -0.058189034 100.0
(1888, 1888, 3)
A : -7.4481964 7.842064
B : -5.883026 10.34

RGB compressed : 0.0 1.0
image is padded
(1888, 1888, 3)
padded_RGB: 0.0 1.0
image converted to LAB
padded_LAB: -0.13128817 100.0
(1888, 1888, 3)
A : -11.879921 13.004303
B : -16.804886 18.523026
X : 0.0012931824 100.0
image processed
RGB compressed : 0.0 1.0
image is padded
(1888, 1888, 3)
padded_RGB: 0.0 1.0
image converted to LAB
padded_LAB: -0.15610754 100.0
(1888, 1888, 3)
A : -14.656067 17.829895
B : -19.981766 20.7798
X : 0.0 100.0
image processed
RGB compressed : 0.0 1.0
image is padded
(1888, 1888, 3)
padded_RGB: 0.0 1.0
image converted to LAB
padded_LAB: -0.19722283 100.0
(1888, 1888, 3)
A : -20.21408 23.61393
B : -25.244522 25.11406
X : 0.0 100.0
image processed
RGB compressed : 0.0 1.0
image is padded
(1888, 1888, 3)
padded_RGB: 0.0 1.0
image converted to LAB
padded_LAB: -0.15506744 100.0
(1888, 1888, 3)
A : -14.120102 14.030457
B : -19.848633 18.536377
X : 0.0 100.0
image processed
RGB compressed : 0.0 1.0
image is padded
(1888, 1888, 3)
padded_RGB: 0.0 1.0
image converted

RGB compressed : 0.0 1.0
image is padded
(1888, 1888, 3)
padded_RGB: 0.0 1.0
image converted to LAB
padded_LAB: -0.14711916 100.0
(1888, 1888, 3)
A : -11.292458 15.189171
B : -18.831253 17.351913
X : 0.0 100.0
image processed
RGB compressed : 0.0 1.0
image is padded
(1888, 1888, 3)
padded_RGB: 0.0 1.0
image converted to LAB
padded_LAB: -0.15600622 100.0
(1888, 1888, 3)
A : -13.421059 23.018837
B : -19.968796 24.559402
X : 0.0 100.0
image processed
RGB compressed : 0.0 1.0
image is padded
(1888, 1888, 3)
padded_RGB: 0.0 1.0
image converted to LAB
padded_LAB: -0.0500381 100.0
(1888, 1888, 3)
A : -6.4048767 16.652107
B : -5.018997 17.368317
X : 0.0 100.0
image processed
RGB compressed : 0.0 1.0
image is padded
(1888, 1888, 3)
padded_RGB: 0.0 1.0
image converted to LAB
padded_LAB: -0.14992356 100.0
(1888, 1888, 3)
A : -15.291214 13.99231
B : -19.190216 19.763565
X : 0.0 100.0
image processed
RGB compressed : 0.0 1.0
image is padded
(1888, 1888, 3)
padded_RGB: 0.0 1.0
image converted to LAB

In [11]:
def mode(): 
    model1 = Sequential([
        #encoder
        Conv2D(128, 2, strides = 2, padding = "same", activation = "relu", input_shape= (1888, 1888, 1)), #testing shape changes
        Conv2D(256, 2, strides = 2, padding = "same", activation = "relu"), 
        Conv2D(256, 2, strides = 2, padding = "same", activation = "relu"), #first param to 512

        #decoder
        Conv2D(256, 2, padding = "same", activation = "relu"), 
        UpSampling2D(2),
        Conv2D(128, 2, padding = "same", activation = "relu"), 
        UpSampling2D(2), 
        Conv2D(2, 2, padding = "same", activation = "tanh"),
        UpSampling2D(2),
    ])
    model1.compile(optimizer ="adam", loss = "mse", metrics = ["accuracy"])
    return model1

In [12]:
model1 = mode()
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 944, 944, 128)     640       
                                                                 
 conv2d_1 (Conv2D)           (None, 472, 472, 256)     131328    
                                                                 
 conv2d_2 (Conv2D)           (None, 236, 236, 256)     262400    
                                                                 
 conv2d_3 (Conv2D)           (None, 236, 236, 256)     262400    
                                                                 
 up_sampling2d (UpSampling2D  (None, 472, 472, 256)    0         
 )                                                               
                                                                 
 conv2d_4 (Conv2D)           (None, 472, 472, 128)     131200    
                                                        

In [13]:
len(X_train)

137

In [14]:
len(Y_train)

137

In [15]:
train_data = tf.data.Dataset.from_tensor_slices((X_train, Y_train))

In [16]:
epochs = 1
history = model1.fit(train_data, batch_size = batch_size, epochs = epochs, verbose = 2)  # add validation_split = 0.2

137/137 - 444s - loss: 3.5649 - accuracy: 0.7688 - 444s/epoch - 3s/step


In [17]:
filename = 'my_model_tf_Y'
model1.save(filename)

INFO:tensorflow:Assets written to: my_model_tf_Y\assets


## Test one image

In [ ]:
#import model

In [ ]:
# load the model from disk
#reconstructed_model = keras.models.load_model("my_model_tf")

In [ ]:
test_path = "C:/Study/Semester2/Machine Learning/ML - Yes Photos/1900s_loose_bw_0036_mom.jpg"

In [ ]:
def prep_img_test(img_path):
    imgRGB = cv2.imread(img_path)
    imgBGR = np.flip(imgRGB, -1)
    
    #check the image category by 'shape' function
    image_size_category = shape(imgBGR) #imgRGB
    #print("image size determined!")

    if image_size_category <1:
        raise Exception("Your image is too small to be used")
    elif image_size_category >4:
        raise Exception("Your image is too big to be used")

    #print("image PCA started!")
    #image_pca = pca_rgb(imgRGB, image_size_category) 
    RGB_compressed = pca_rgb(imgBGR, image_size_category) #imgRGB, image_size_category
    #RGB_compressed[400][400]
    #plt.figure(figsize=(10, 10))
    #plt.imshow(RGB_compressed)
    
    print("image is padded")
    padded_image = rgb_pad(RGB_compressed)
    #padded_image[1000][1000]
    #plt.figure(figsize=(10, 10))
    #plt.imshow(padded_image)
    
    #image_pca = np.float32((1.0/255)*image_pca) # #The rgb_compressed come out in RGB scaled data already
    print("image converted to LAB")
    padded_LAB = cv2.cvtColor(padded_image, cv2.COLOR_RGB2Lab) #convert padded RGB compressed to LAB
    padded_LAB = padded_LAB/np.amax(padded_LAB) #scaled to -1/1
    #imgLAB = np.clip(imgLAB, 0, 1)

    
    #separate L and AB color channels
    Y = padded_LAB[:, :,1:]
    #Y = Y / 128 #comment out when show predicted
    #Y = Y.tolist()
    X = padded_LAB[:, :, 0] #.tolist()
    #print(X[1000])
    #plt.figure(figsize=(10, 10))
    #plt.imshow(X)
    
    #this is in sample code and idk why
    
    X = X.reshape(1, 1888, 1888, 1) #comment out when show predicted
    Y = Y.reshape(1, 1888, 1888, 2) #comment out when show predicted
    
    print("image processed")

    return Y, X 

In [ ]:
CL, BW = prep_img_test(test_path)
#test_data = tf.data.Dataset.from_tensor_slices(BW)
test_predict = model1.predict(BW) #what shape does the prediction takes?

In [ ]:
type(test_predict)

In [ ]:
test_predict.shape # use this to merge

In [ ]:
BW.shape # use this to merge

In [ ]:
CL.shape

#### merge with d stack

In [ ]:
test_predict_reshaped = test_predict[0,:,:,:]

In [ ]:
test_predict_reshaped.shape

In [ ]:
test_predict_reshaped[0][0]

In [ ]:
plt.imshow(test_predict_reshaped[:,:,0])

In [ ]:
plt.imshow(test_predict_reshaped[:,:,1])

In [ ]:
BW_reshaped  = BW[0,:,:,:]

In [ ]:
BW_reshaped.shape

In [ ]:
plt.imshow(BW_reshaped)

In [ ]:
test_merged_LAB = np.dstack((BW_reshaped, test_predict_reshaped))

In [ ]:
test_merged_LAB.shape

In [ ]:
test_merged_LAB[0][0]

In [ ]:
plt.imshow(test_merged_LAB)

In [ ]:
#test_merged_converted = color.Lab2rgb(test_merged)
test_merged_rgb = test_merged_rgb*128

In [ ]:
plt.imshow(test_merged_rgb)

In [ ]:
test_merged_rgb = cv2.cvtColor(test_merged, cv2.COLOR_Lab2RGB) 

In [ ]:
test_merged_rgb[0][0] # this is weird

In [ ]:
#test_merged_rgb_scaled = (test_merged_rgb/255).astype('float32')

In [ ]:
#plt.imshow(test_merged_rgb)
plt.imshow(test_merged_rgb_scaled )

In [ ]:
plt.imsave('result1.jpg', test_merged_rgb)

##### a way to merge the prediction

In [ ]:
#pastes predicted AB to RGB blank canvas
canvas = np.zeros((1888,1888,3))
canvas[:,:,0] = BW[0][:,:,0]
canvas[:,:,1:] =test_predict[0]

In [ ]:
#convert AB (in RGB space) to rgb photo
rgb_converted = color.lab2rgb(canvas)

In [ ]:
plt.imshow(rgb_converted)

In [ ]:
lab_2rgb = cv2.cvtColor(canvas, cv2.COLOR_RGB2Lab)

In [ ]:
lab_to = 

In [ ]:
lab_merge.shape

In [ ]:
plt.imshow(canvas)

In [ ]:
#Test input array

In [ ]:
img_path1 = 'C:/Study/Semester2/Machine Learning/ML - Yes Photos/1900s_loose_bw_0036_mom.jpg'
Y01, X01 = prep_img(img_path1)

In [ ]:
type(X01[0])

In [ ]:
#show AB channel of the image
plt.imshow(X01) #X01[0,:,:,0]

In [ ]:
df_X01 = pd.DataFrame(X01[0,:,:,0])

In [ ]:
df_Y01 = pd.DataFrame(Y01[0,:,:,0])

In [ ]:
df_Y01.describe()